In [1]:
from main import load_and_preprocess_data

df, encoders = load_and_preprocess_data()

/Users/heifen/Documents/studie/machinelearning/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading and preprocessing data...
Class distribution:
is_fraud
0    1289169
1       7506
Name: count, dtype: int64
Fraud percentage: 0.58%


In [2]:
from model import load_lstm_model
from predict import predict_and_analyze
from main import SEQUENCE_LENGTH

# sample_cc_num = df['cc_num'].iloc[0]
sample_cc_num = 2703186189652095
user_transactions = df[df['cc_num'] == sample_cc_num]

last_transaction = user_transactions.iloc[-1].to_dict()

lstm_model, scaler, processor = load_lstm_model()

results = predict_and_analyze(
    lstm_model,
    user_transactions.iloc[:-1],  # Use all but last transaction
    SEQUENCE_LENGTH,
    processor,
    actual_transaction=last_transaction
)

print("\nPredicted most likely values for next transaction:")
print("---------------------------------------------")
for feature, value in results['prediction']['most_likely_values'].items():
    print(f"{feature:20}: {value}")

Loading model from models/lstm_transaction_model.h5...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step

Predicted most likely values for next transaction:
---------------------------------------------
amt                 : 119.68
lat                 : 47.354200000000006
long                : -95.24744999999999
merch_lat           : 47.50744976666667
merch_long          : -95.32028693333334
hour                : 0
day                 : 6
month               : 10
dayofweek           : 3
merchant_encoded    : 202
category_encoded    : 2


In [15]:
last_transaction

{'Unnamed: 0': 1295986,
 'trans_date_trans_time': Timestamp('2020-06-21 06:08:15'),
 'cc_num': 2703186189652095,
 'merchant': 'fraud_Jaskolski-Vandervort',
 'category': 'misc_net',
 'amt': 60.48,
 'first': 'Jennifer',
 'last': 'Banks',
 'gender': 'F',
 'street': '561 Perry Cove',
 'city': 'Moravian Falls',
 'state': 'NC',
 'zip': 28654,
 'lat': 36.0788,
 'long': -81.1781,
 'city_pop': 3495,
 'job': 'Psychologist, counselling',
 'dob': Timestamp('1988-03-09 00:00:00'),
 'trans_num': 'e56607c5a11ee89c873fbc0e402b5df3',
 'unix_time': 1371794895,
 'merch_lat': 35.376104,
 'merch_long': -82.162645,
 'is_fraud': 0,
 'hour': 6,
 'day': 21,
 'month': 6,
 'dayofweek': 6,
 'age': 37,
 'merchant_encoded': 281,
 'category_encoded': 8,
 'gender_encoded': 0,
 'city_encoded': 526,
 'state_encoded': 27,
 'job_encoded': 370}

In [14]:
results

{'prediction': {'most_likely_values': {'amt': np.float64(119.68),
   'lat': np.float64(47.354200000000006),
   'long': np.float64(-95.24744999999999),
   'merch_lat': np.float64(47.50744976666667),
   'merch_long': np.float64(-95.32028693333334),
   'hour': np.int64(0),
   'day': np.int64(6),
   'month': np.int64(10),
   'dayofweek': np.int64(3),
   'merchant_encoded': np.int64(202),
   'category_encoded': np.int64(2)},
  'raw_distributions': {'amt': array([[0.01589074, 0.0168743 , 0.02078583, 0.02280238, 0.03648099,
           0.01857636, 0.03135741, 0.0226029 , 0.02214682, 0.0180739 ,
           0.02334327, 0.0321891 , 0.02773414, 0.02339871, 0.00893579,
           0.03012874, 0.0426755 , 0.01856796, 0.01788842, 0.03303154,
           0.01216083, 0.01692942, 0.01727447, 0.02302742, 0.01256863,
           0.03086864, 0.02065773, 0.01697345, 0.018658  , 0.02568389,
           0.01290507, 0.01765598, 0.01187358, 0.01772085, 0.01305088,
           0.01859118, 0.00682185, 0.01794682, 0.01